In [50]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
import json_lines, json, copy

In [51]:
#Remove '/project1' for the path

from pathlib import Path

raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
if raw_instance_filePath.is_file():
    print("has")
raw_instance_dir = Path("../clickbait_dataset")

#If you want to check the diretory by using .is_dir(), then you need to make another variable.
if raw_instance_dir.is_dir():
    print("has, too")
else:
    print("none")

has
has, too


In [52]:
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
if raw_truth_filePath.is_file():
    print("has")
raw_truth_dir = Path("../clickbait_dataset")
if raw_truth_dir.is_dir():
    print("has, too")
else:
    print("none")

has
has, too


In [53]:
#open with json - raw_instance_train
import json
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[1]


{'targetCaptions': ['Stocks Fall as Investors Watch Central Banks',
  'Do Traders See Data Before Release? Markets Say Yes',
  'Be Careful: Stock Volatility is Hiding, Not Hibernating',
  'Singapore Bans Ex-Goldman Banker Linked to 1MDB From Working in Financial Sector',
  'Demanding and Direct: HSBC’s New Chairman Mark Tucker',
  'Saudi King’s Asia Expedition Enters Crucial Phase',
  'Puerto Rico Board Approves Fiscal Road Map',
  'CFPB, Justice Department Poised to Square Off in Court',
  'Raphael Bostic to Lead Atlanta Fed',
  'Blackstone Sells Part of Stake in NCR',
  'Dartmouth College Appoints Alice Ruth as New Endowment Chief',
  'Hedge-Fund Loses Big on Oil Bets',
  'Buyout Firm Buys $800 Million of Assets From Itself',
  'J.P. Morgan Moves Ahead With Plan to Drop Commissions in IRAs',
  'Fed Signal Could Revive a Problem for China’s Central Bank',
  'Why Apple and Pfizer Are Piling Into Taiwan’s Bond Market',
  '[https://m.wsj.net/video/20170312/031017vinny/031017vinny_167x94.

In [54]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'truthMedian': 0.33333334,
 'truthMean': 0.2,
 'truthMode': 0.33333334,
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'id': '7'}

In [55]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [56]:
raw_instance_train = [json.loads(line) for line in open("../project1/dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../project1/dataset/truth_train.jsonl")]

FileNotFoundError: [Errno 2] No such file or directory: '../project1/dataset/instances_train.jsonl'

In [57]:
#To Cleaning data. Later...

# import re

# def cleanhtml(raw_html):
#   cleanr = re.compile('<.*?>')
#   cleantext = re.sub(cleanr, '', raw_html)
#   return cleantext

In [58]:
# cleanhtml(str)

In [60]:
# #separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        non_clickbait_train.append(raw_instance_train[x])

In [15]:
#clickbait_train
clickbait_train[0]

{'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'postMedia': [],
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns about the star goalkeeper.",
  "The

In [16]:
#non_clickbait_train
non_clickbait_train[0]

{'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
  "The guidelines

In [49]:
from nltk.tokenize import word_tokenize
from string import punctuation

tokensList = []

for x in range(0, len(raw_instance_train)):
    tokens = word_tokenize(raw_instance_train[x]['postText'][0])
    tokensList.append(tokens)
    tokensList[x] = [c for c in tokens if c not in punctuation]
    
print(tokensList[0])
len(tokensList[0])

['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost']


15